In [1]:
import numpy as np
import pandas as pd
import ast
import itertools
import collections
import struct
import sys
from sympy import poly, var
from collections import Counter

In [2]:
col_names  = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']
row_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f']

enc_key_sbox = pd.read_excel('AES tables.xlsx', index_col=0)

enc_key_sbox.columns = col_names ## replacing the column names 
enc_key_sbox.index = row_names    ## replacing the row names

enc_key_sbox.head()

,0,1,2,3,4,5,6,7,8,9,a,b,c,d,e,f
0,63,7c,77,7b,f2,6b,6f,c5,30,1,67,2b,fe,d7,ab,76
1,ca,82,c9,7d,fa,59,47,f0,ad,d4,a2,af,9c,a4,72,c0
2,b7,fd,93,26,36,3f,f7,cc,34,a5,e5,f1,71,d8,31,15
3,4,c7,23,c3,18,96,5,9a,7,12,80,e2,eb,27,b2,75
4,9,83,2c,1a,1b,6e,5a,a0,52,3b,d6,b3,29,e3,2f,84


## key Generation

In [3]:
def key_sbox(element):
    row_index = element[0]
    col_index  = element[1]
    ans = enc_key_sbox.loc[[row_index],[col_index]][col_index][0]
    return str(ans)

In [4]:
def binaryToDecimal(binary):
    binary1 = binary
    decimal, i, n = 0, 0, 0
    
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)
        binary = binary//10
        i += 1
        
    hexadecimal =  hex(decimal)[-1]
    return hexadecimal

In [5]:
def bcd_hexadecimal(bcd_list):
    w4 = []
    
    for i in range(0,len(bcd_list),2):
        temp_str = ' '
        temp_str = binaryToDecimal(int(bcd_list[i]))
        temp_str = temp_str + binaryToDecimal(int(bcd_list[i+1]))
        w4.append(temp_str)
        
    return w4

In [6]:
def HexaDecimaltoBCD(str):
    list1 = []
    for i in range(len(str)):
        decimal = int(str[i], 16)                                ## hexadecimal -> decimal
        binary_num = bin(decimal).replace("0b", "")  # decimal -> binary
        list1.append(binary_num)
            
    ## binary in-terms of 4 bits
    for i in range(len(list1)):
        element = list1[i]
        if len(element)<4:
            diff = 4 - len(element)
            for j in range(diff):
                element = "0" + element
            list1[i] = element
            
    return list1

In [7]:
def Hexaword_BCD(list1):
    bcd = [ HexaDecimaltoBCD(i) for i in list1 ]
    bcd = list(np.concatenate(bcd).flat)    ## 2d list to 1d list
    return bcd

In [8]:
def XOR(list1, list2):
    
    def compare(element1, element2):
        ans = []
        for i,j in zip(element1,element2):
            if (i!=j):ans.append(1)
            else:ans.append(0)
        return ans

    main_ans = []
    for i in range(len(list1)):
        main_ans.append(compare(list1[i], list2[i]))
        
    main_ans = [ "".join(list(map(str, i))) for i in main_ans ]
    return main_ans

In [9]:
# Function to convert BCD to hexadecimal
def bcdToHexaDecimal(s):
    len1 = len(s)
    check = 0
    num = 0
    sum = 0
    mul = 1
    ans = []

# Iterating through the bits backwards
    i = len1 - 1
     
    while(i >= 0):
        sum += (ord(s[i]) - ord('0')) * mul
        mul *= 2
        check += 1
 
        # Computing the hexadecimal number formed
        # so far and storing it in a vector.
        if (check == 4 or i == 0):
             
            if (sum <= 9):
                ans.append(chr(sum + ord('0')))
            else:
                ans.append(chr(sum + 55));
 
            # Reinitializing all variables for next group.
            check = 0
            sum = 0
            mul = 1
        i -= 1
         
    len1 = len(ans)
 
    # Printing the hexadecimal
    # number formed so far.
    i = len1 - 1
     
    while(i >= 0):
        return ans

In [10]:
def rotword(word):
    rot=word[1:]
    rot.append(word[0])
    return rot

In [11]:
def col1_generation(iteration_var, hex_key):
    
    ## round-constant table 
    ## key-round and value-hexadecimal
    round_constant = {1:'1', 2:'2', 3:'4', 4:'8', 5:'10', 6:'20', 7:'40', 8:'80', 9:'1B', 10:'36'}

    ## taking the last column words
    last_col = hex_key[-1]

    ## left shifting the last column words
    #left_shift = last_col[1:] + last_col[:1] 
    left_shift = rotword(last_col)

    ## sub-word generation from S-Box
    subword = []
    for i in left_shift:
        val = key_sbox(i)
        if len(val)!=2:
            val = '0' + val
            subword.append(val)
        else:
            subword.append(val)

    ## subword --> hexadecimal(subword)
    y1 = Hexaword_BCD(subword)

    ## subword (XOR) Round Constant
    element = round_constant[iteration_var]
    initial = HexaDecimaltoBCD(element)
    initial_length = len(initial)

    if (initial_length == 1):
        between = initial
        temp_1 = [ [0 for j in range(4) for i in range(1)] ]
        last = [ [0 for j in range(4)] for i in range(6) ]
        temp_1.extend(between)
        temp_1.extend(last)
        final = temp_1
        if (element==1): ## if there is a single element(0,1,2, ... ,9) from the s-box --> length=1
            before1 = [ [0 for j in range(4) for i in range(1)] ]
            before1.extend(final)
            final = before1

    elif (initial_length == 2):  ## if there is a two element(11, 1a, b1, ...) from the s-box --> length=2
        last = [ [0 for j in range(4)] for i in range(6) ]
        first = initial
        first.extend(last)
        final = first

    final = [ "".join(list(map(str, i))) for i in final ]
    round_list = final

    round_ans = XOR(y1, round_list) ## --> g(col4_before)
    #print("y1 : ",y1)
    # taking col1_before
    col1_before = [ HexaDecimaltoBCD(i) for i in hex_key[0] ]
    col1_before = list(np.concatenate(col1_before).flat)

    ## col1 --> col1_before (exor) g(col4_before)
    col1 = XOR(col1_before, round_ans)
    col1 = bcd_hexadecimal(col1)

    return col1

In [12]:
def key_generation(hex_key):
    complete_keys = []
    for i in range(1,11): ## 10 times running
        temp = []

        ## 1st col word = col_before[0] (exor) g(col_before[-1]) 
        col1_hex = col1_generation(i, hex_key)
        col1_bin  = Hexaword_BCD(col1_hex)

        for j in range(2,5): ## each loop, 3 times running ( remaining 3 words)
            if (j==2):  ## 2nd col words
                col1_before_bin = Hexaword_BCD(hex_key[j-1])
                col2_bin = XOR(col1_before_bin, col1_bin)
                col2_hex = bcd_hexadecimal(col2_bin)

            else:  ## 3rd and 4th column words
                col2_before_bin = Hexaword_BCD(hex_key[j-1])            
                col3_bin = XOR(col2_before_bin, col2_bin)
                col3_hex = bcd_hexadecimal(col3_bin)
                col2_bin = Hexaword_BCD(col3_hex)
                temp.append(col3_hex)

        ## once again generating the hex_key
        hex_key = []

        hex_key.append(col1_hex)
        hex_key.append(col2_hex)
        hex_key.extend(temp)    

        complete_keys.append(hex_key)
    return complete_keys

## Round Operation

In [13]:
def add_round_key(hex_str, hex_key): ## exor with plain text and key
    
    add_round_key = []
    for i in range(4):
        bin_str = Hexaword_BCD(hex_str[i])
        bin_keys = Hexaword_BCD(complete_keys[0][i])
        bin_xor   = XOR(bin_str, bin_keys)
        bcd_round_key = bcd_hexadecimal(bin_xor)
        add_round_key.append(bcd_round_key)

        return add_round_key

In [14]:
def subtitute_box(add_round_key):  ## output from add_round_key
    subword = []
    for i in range(4):
        temp_word = []
        for j in add_round_key:
            val = key_sbox(j[i])
            if len(val)!=2:
                val = '0' + val
                temp_word.append(val)
            else:
                temp_word.append(val)

        ## subword --> hexadecimal(subword)
        temp_word = Hexaword_BCD(temp_word)
        temp_word_hexa = bcd_hexadecimal(temp_word)
        subword.append(temp_word_hexa)
        temp_word_hexa = []

    ## swapping rows and columns
    subword = [list(x) for x in zip(*subword)]
    return subword

In [15]:
def shift_rows(subword):
    shift_rows = []
    subword = collections.deque(subword)
    for i in range(4):
        temp = collections.deque(subword[i])
        temp.rotate(-i)
        shift_rows.append(list(temp))
    return shift_rows

In [16]:
def mix_columns_each_round(multiple, shift_rows):
    prod_ans1 = []
    row_ans = []
    final_ans  = []

    for i in range(4):
        for j in range(4):
            for k in range(4):

                ## hexadecimal to BCD
                num1 = Hexaword_BCD(multiple[i][k])
                num2 = Hexaword_BCD(shift_rows[j][k])
                print("\nHexadecimal Format : num-1 : {} and num-2 : {} ".format(multiple[i][k], shift_rows[j][k]))
                print("BCD Format : num-1 : {} and num-2 : {} ".format(num1, num2))

                ## Product operation
                num1_list = [int(item) for sublist in num1 for item in sublist]
                num2_list = [int(item) for sublist in num2 for item in sublist]
                ans = list(np.poly1d(num1_list) * np.poly1d(num2_list))
                print("Product value : {}".format(ans))
                prod_ans1.append(ans)

            print("\n")
            max_length = max([len(p) for p in prod_ans1])

           ## adding 0's 
            temp_match_len = []
            for a in prod_ans1:
                for b in range(max_length - len(a)):
                    a.insert(0, 0)
                element = ''.join(map(str, a))
                binary_to_polynomial(element)
                temp_match_len.append(a)
            prod_ans1 = temp_match_len

            ## summing up the polynomials
            temp_sum = []
            for c in range(max_length):
                temp_sum.append(sum([sub[c] for sub in prod_ans1]))
            for d in range(len(temp_sum)):
                if (temp_sum[d] %2!= 0): temp_sum[d] = 1  ## sum is odd number put 1
                else:  temp_sum[d] = 0                               ## sum is even number put 0
            prod_ans1 = temp_sum

            prod_ans1= ''.join(map(str, prod_ans1))

            print("Summed up value : ",prod_ans1)
            binary_to_polynomial(prod_ans1)

            if len(prod_ans1)>8:
                irreducible_polynomial = '100011011'
                ir_ans = binary_division_modulo_2(prod_ans1, irreducible_polynomial)
                prod_ans1= ir_ans
                print("ir --> ans : ",ir_ans)
                hexadecimal_ans = bcd_to_hexadecimal(prod_ans1)
                row_ans.append(hexadecimal_ans)
                print("FINAL : ",hexadecimal_ans)
            else:
                hexadecimal_ans = bcd_to_hexadecimal(prod_ans1)
                row_ans.append(hexadecimal_ans)
                print("FINAL : ",hexadecimal_ans)
            prod_ans1 = []
        final_ans.append(row_ans)
        row_ans = []
    return final_ans

In [17]:
def main():
    round_num = 0
    add_round_key
    subtitute_box
    shift_rows
    mix_columns_each_round

## Getting the user inputs

In [18]:
def text_hexadecimal(text): ## all the blocks --> 16bytes
    hex_text   = []
    for i in text: ## character -> ascii (decimal) -> hexa-decimal
        hex_text.append(hex(ord(i))[2:])
    return hex_text ## 16-byte representation of the text

In [19]:
plain_text = "Two One Nine Two"
plain_key = 'Thats my Kung Fu' 

hex_str = text_hexadecimal(plain_text)
hex_key = text_hexadecimal(plain_key)

print("Text Hexadecimal Representation : ",hex_str)
print("Key Hexadecimal Representation : ",hex_key)

Text Hexadecimal Representation :  ['54', '77', '6f', '20', '4f', '6e', '65', '20', '4e', '69', '6e', '65', '20', '54', '77', '6f']
Key Hexadecimal Representation :  ['54', '68', '61', '74', '73', '20', '6d', '79', '20', '4b', '75', '6e', '67', '20', '46', '75']


In [20]:
## splitting into 4*4 matrix
hex_key = [hex_key[i:i+4] for i in range(0, len(hex_key), 4)]
hex_key

[['54', '68', '61', '74'],
 ['73', '20', '6d', '79'],
 ['20', '4b', '75', '6e'],
 ['67', '20', '46', '75']]

In [21]:
complete_keys = key_generation(hex_key)
for i in range(len(complete_keys)):
    print("Round - {} keys --> ".format(i+1),end=' ')
    print(*list(itertools.chain(*complete_keys[i])))

Round - 1 keys -->  e2 32 fc f1 91 12 91 88 b1 59 e4 e6 d6 79 a2 93
Round - 2 keys -->  56 08 20 07 c7 1a b1 8f 76 43 55 69 a0 3a f7 fa
Round - 3 keys -->  d2 60 0d e7 15 7a bc 68 63 39 e9 01 c3 03 1e fb
Round - 4 keys -->  a1 12 02 c9 b4 68 be a1 d7 51 57 a0 14 52 49 5b
Round - 5 keys -->  b1 29 3b 33 05 41 85 92 d2 10 d2 32 c6 42 9b 69
Round - 6 keys -->  bd 3d c2 87 b8 7c 47 15 6a 6c 95 27 ac 2e 0e 4e
Round - 7 keys -->  cc 96 ed 16 74 ea aa 03 1e 86 3f 24 b2 a8 31 6a
Round - 8 keys -->  8e 51 ef 21 fa bb 45 22 e4 3d 7a 06 56 95 4b 6c
Round - 9 keys -->  bf e2 bf 90 45 59 fa b2 a1 64 80 b4 f7 f1 cb d8
Round - 10 keys -->  28 fd de f8 6d a4 24 4a cc c0 a4 fe 3b 31 6f 26


In [23]:
def add_round_key(hex_str, hex_key): ## exor with plain text and key
def subtitute_box(add_round_key):  ## output from add_round_key
def shift_rows(subword):
def mix_columns_each_round(multiple, shift_rows):

[['e2', '32', 'fc', 'f1'],
 ['91', '12', '91', '88'],
 ['b1', '59', 'e4', 'e6'],
 ['d6', '79', 'a2', '93']]